# RandomForest

In [1]:
#SetupCode

import csv
import numpy
import pandas as pd
from matplotlib import pyplot
from sklearn.preprocessing import Binarizer
from numpy import set_printoptions
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import pydot

''' Load data and create a dataframe '''

filename = 'train_imperson_without4n7_balanced_data.csv'

df = pd.read_csv('C:\\Users\\timot\Desktop\\AMLGroupProject\\train_imperson_without4n7_balanced_data.csv', header=0, na_values=['nan'])
print(df.head())
print(df.info())
print(df.describe())

''' Delete columns with no values '''

# many features have no values
columnsToDelete = []

# if the  feature has std == 0 it means no values or only one value

for key, value in df.iteritems():
    if df[key].std() == 0:
        columnsToDelete.append(key)

df = df.drop(columnsToDelete, axis=1)
len(columnsToDelete)  # 74 features have been deleted

print(df.head())


###Using the RF

df.columns=[i for i in range(1,80)] #rename column heads

X = df.iloc[:, 0:78] #set x variables to be the first 78 columns
y = df.iloc[:,-1] #set y variable to be last column
print(y.head())

columnnames = list(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25) #75% training data, 25% test data

   1  2  3         5         6         8         9  10  11  12  ...  146  147  \
0  0  0  0  0.000066  0.000066  0.009150  0.009150   0   0   0  ...  0.0    0   
1  0  0  0  0.000014  0.000014  0.000000  0.000000   0   0   0  ...  0.0    0   
2  0  0  0  0.035528  0.035528  0.070588  0.070588   0   0   0  ...  0.0    0   
3  0  0  0  0.005128  0.005128  0.094771  0.094771   0   0   0  ...  0.0    0   
4  0  0  0  0.035116  0.035116  0.070588  0.070588   0   0   0  ...  0.0    0   

   148  149  150  151  152  153  154  155  
0    0    0    0    0    0    0  0.0    0  
1    0    0    0    0    0    0  0.0    0  
2    0    0    0    0    0    0  0.0    0  
3    0    0    0    0    0    0  0.0    0  
4    0    0    0    0    0    0  0.0    0  

[5 rows x 153 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97044 entries, 0 to 97043
Columns: 153 entries, 1 to 155
dtypes: float64(48), int64(105)
memory usage: 113.3 MB
None
             1        2        3             5            

# Randomforest model creation and prediction


In [2]:
#Create model with 1000 trees to start. This includes bagging. Set bootstrap = False to turn off

randf = RandomForestClassifier(n_estimators = 100, random_state = 22)
randf.fit(X_train, y_train)

yhat = randf.predict(X_test)

# Metrics and variable importance


In [3]:
print(confusion_matrix(y_test,yhat))
print(classification_report(y_test,yhat))

#It's perfect...

# Get numerical features importance and create a tuple list with the columnnames.
importances = list(randf.feature_importances_)

feature_importances = [(feature,round(importance,2)) for feature,importance in zip(columnnames ,importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];


#Nothing really stands out....

[[12250     0]
 [    0 12011]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12250
           1       1.00      1.00      1.00     12011

    accuracy                           1.00     24261
   macro avg       1.00      1.00      1.00     24261
weighted avg       1.00      1.00      1.00     24261

Variable:                   23 Importance: 0.11
Variable:                   33 Importance: 0.11
Variable:                    2 Importance: 0.09
Variable:                   12 Importance: 0.07
Variable:                    1 Importance: 0.05
Variable:                    3 Importance: 0.05
Variable:                    4 Importance: 0.05
Variable:                   30 Importance: 0.05
Variable:                   14 Importance: 0.03
Variable:                   17 Importance: 0.03
Variable:                   21 Importance: 0.03
Variable:                   35 Importance: 0.03
Variable:                   71 Importance: 0.03
Variable:         

#  Let's limit the depth...


In [4]:
# Max depth is reduced to 3

randfs = RandomForestClassifier(n_estimators = 100, random_state = 22, max_depth=3 )
randfs.fit(X_train, y_train)

yhats = randfs.predict(X_test)

print(confusion_matrix(y_test,yhats))
print(classification_report(y_test,yhats))

importances2 = list(randfs.feature_importances_)

feature_importances2 = [(feature,round(importance,2)) for feature,importance in zip(columnnames ,importances2)]
feature_importances2 = sorted(feature_importances2, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances2];




[[12130   120]
 [   15 11996]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     12250
           1       0.99      1.00      0.99     12011

    accuracy                           0.99     24261
   macro avg       0.99      0.99      0.99     24261
weighted avg       0.99      0.99      0.99     24261

Variable:                   23 Importance: 0.12
Variable:                   33 Importance: 0.12
Variable:                    2 Importance: 0.09
Variable:                   12 Importance: 0.06
Variable:                    1 Importance: 0.05
Variable:                    3 Importance: 0.05
Variable:                   14 Importance: 0.05
Variable:                   30 Importance: 0.05
Variable:                   17 Importance: 0.04
Variable:                   71 Importance: 0.04
Variable:                    4 Importance: 0.03
Variable:                   16 Importance: 0.03
Variable:                   21 Importance: 0.03
Variable:         

Accuracy is significantly reduced. Importance of features not great....
